#### Part 1 - Extract the "Relevant Text" from pdf

##### Here we find keywords in text such as : Goal/Goals, Targets, Objective.  By doing this we directly reduce pages from around 100 to 30. So our search scope is narrowed down

In [124]:
import PyPDF2
import string

In [139]:
# Open the PDF
with open('Fraport-2021.pdf', 'rb') as file:
    pdf = PyPDF2.PdfFileReader(file)
    pdftext=''
    # Iterate through each page of the PDF
    for page in range(pdf.getNumPages()):
        text = pdf.getPage(page).extractText()
        # Search for goals and targets
        if 'goal' in text or 'target' in text or 'objective' in text.lower():
            #Removing special characters
            filtered_text = ''.join(filter(lambda x:x in string.printable, text))
            # removeing unwanted spaces
            filtered_text = filtered_text.replace('\n', ' ').replace('\r', '').replace('\t', '')
            pdftext = pdftext+ "\n \n" +filtered_text
    print(pdftext)



 
Five dimensions with fourteen thematic fields form the  framework for action for Fraport We understand sustainability as the interplay of  ecological, economic and social aspects.  We have defined concrete targets for the five  dimensions derived from this and report annually  on their measures and target achievement. 6 Employees Economic  enhancement CommunityGovernanceEcologyEconomy Social Environment
 
Environmental Goals Focus on climate protection: Our contribution to a climate -neutral  Europe 9No offsetting of CO 2emissions (Scope 1 and 2) is to take place throughout the Group. We want to meet people's need for mobility without placing an additional burden on the environment. We  are working at full speed to achieve this. Climate protection is one of the most urgent tasks facing the entire  aviation industry. That is why we have once again tightened our CO2 targets in 2021.  New targets ofFraport AG In thousand t CO 2New targets forFraport Group In thousand t CO 2 229 Base ye

### Part2 : Fine Tune Transformer model

In [1]:
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification
import tensorflow as tf

c:\Users\nikhi\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


##### Setup Bert   

##### Tokenizer

In [2]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

c:\Users\nikhi\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:129: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nikhi\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [102]:
import pandas as pd

In [103]:
df = pd.read_csv("goaldata.csv", sep=";",encoding="UTF-8-SIG")

In [104]:
df.columns

Index(['Label', 'Text'], dtype='object')

In [105]:
df.Text[0]

'Reduce carbon emissions by 50% by 2030'

In [128]:
## This number needs to ne changed based on available data, It should be arounf 70-805 of the main data

training_size = 90

In [107]:
sentences = []
labels = []
for x in range(0,len(df)):
    sentences.append(df.Text[x])
    labels.append(df.Label[x])

In [108]:
training_sentences = sentences[0:training_size]
validation_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
validation_labels = labels[training_size:]

In [109]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [110]:
train_encodings = tokenizer(training_sentences,
                            truncation=True,
                            padding=True)
val_encodings = tokenizer(validation_sentences,
                            truncation=True,
                            padding=True)

In [111]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    training_labels
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    validation_labels
))

In [112]:
# We classify two labels in this example. In case of multiclass 
# classification, adjust num_labels value
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_projector', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_59']
You should probably TRAIN this model on a down-stream task to be able to use i

In [116]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])
model.fit(train_dataset.shuffle(50).batch(2),
          epochs=3,
          batch_size=2,
          validation_data=val_dataset.shuffle(30).batch(2))

Epoch 1/3
45/45 [==============================] - 42s 678ms/step - loss: 0.0823 - accuracy: 0.9889 - val_loss: 1.4765 - val_accuracy: 0.7429
Epoch 2/3
45/45 [==============================] - 29s 651ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 1.7728 - val_accuracy: 0.7429
Epoch 3/3
45/45 [==============================] - 29s 650ms/step - loss: 7.6462e-04 - accuracy: 1.0000 - val_loss: 1.9696 - val_accuracy: 0.7429


In [145]:
lines= pdftext.splitlines()
lines = [x for x in lines if x.strip()]

In [157]:
predict_input = tokenizer.encode(lines[0],
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")

In [160]:
output = model.predict(predict_input)[0]

1/1 [==============================] - 0s 120ms/step


In [161]:
tf_prediction = tf.nn.softmax(output, axis=1).numpy()[0]

In [163]:
tf_prediction[0]

0.99955183